<div style="text-align: center;">

# **Algoritmia Cuantica** 
# Algoritmo de Deustch-Jozsa 
<font size="2">

##### Daniel Amauri Vázquez Gutiérrez
</font>


**Realiza el Circuito de Deutsch Jozsa generalizado, construyendo una compuerta para n qubits**
**Como datos de entrada se pedira el numero de qbits.** 

Entonces procedemos analizando el circuito a nivel teorico en primer lugar.  

Como sabemos tenemos una entrada de la siguiente forma : 

$$|\Psi\rangle =|0\rangle^{\otimes n}|1\rangle$$

Despues , a cada una de las $n$ entradas se añade una compuerta de Hadamard, con esto obtenemos :  

$$|\Psi\rangle_1=H^{\otimes n}|0\rangle^{\otimes n} \otimes H |1\rangle=\frac{1}{\sqrt{2^n}}\sum^{2^n-1}_{x=0}|x\rangle |-\rangle$$ 

Podemos notar en esta expresion que tenemos la generalizacion con $x$ , representando una cadena de $n$ numeros  entre 0 y 1. Cuando dice que la sumatoria va desde $x=0$ hasta $x=2^n-1$ se refiere a que empieza con la representacion **en binario** de  $x=0$ y llega  hasta  $x=2^n-1$ igualmente representado en binario. 

Sin embargo por practicidad a estas cadenas de numeros las podemos ver como vectores $|2\rangle$,$|3\rangle$,$|4\rangle$, etc. Sin perdida de generalidad  y propiedades; unicamente no olvidando lo que enrealidad estan representando. 

Nuestro siguiente problema de interes es el ver como generar la representacion matricial del oraculo $U_f$. Siguiendo el como se comportaba  el Oraculo  en el algoritmo de Deustch , entonces notamos la siguiente caracteristica : 

$$U_f(|x\rangle |0\rangle)=|x\rangle |0 \oplus f(x) \rangle =|x\rangle |f(x) \rangle $$
$$U_f(|x\rangle |1\rangle)=|x\rangle |1 \oplus f(x) \rangle =|x\rangle |\overline{f(x) }\rangle $$


Esto , mas la propiedad  de la forma matricial de los Operadores que nos dice que :  
$$A_{ij}= \langle u_i | A | u_j\rangle $$

Remarcando en primera instancia que como cada Qbit lo representamos como un vector $1\times 2$ , entonces al ser  un sistema con $n+1$ Qbits , nuestra representacion matricial tendra entradas de $2^{n+1}\times 2^{n+1}$  

Podemos notar que las  columnas de la matriz representan las 
$$|x\rangle|y\rangle$$ 

Con  $x\epsilon N$ y $y\epsilon[0,1]$, por otro lado  las filas las podemos ver como vectores  como el adjunto del anterior, esto es :  
$$\langle y|\langle x|$$ 

De tal forma que para alguna columna o fila $i$ , podemos identificar su vectores $x$ y $y$ de la siguiente forma: 
$$y= mod_2(i)$$  
$$x= \frac{i-y}{2}$$



Juntando estos resultados , junto con nuestras relaciones de estos vectores al ser evaluados en nuestra matriz $U_f$, podemos hacer  como ejemplo las primeras dos matrices , cuando $n=1$ y cuando $n=2$ :  

* **$n=1$** 
$$
\begin{matrix}
1-f(0) & f(0) &0&0 \\
f(0) & 1-f(0) &0&0\\
0&0&1-f(1)&f(1)\\
0&0&f(1)&1-f(1)\\
\end{matrix}
$$  

Notese que entonces tenemos 2 Qbits y que la dimencion de nuestra matris es $2^{1+1}=2^{2}=4$

* **$n=2$** 
$$
\begin{matrix}
1-f(0) & f(0) &0&0&0&0&0&0 \\
f(0) & 1-f(0) &0&0&0&0&0&0\\
0&0&1-f(1)&f(1)&0&0&0&0\\
0&0&f(1)&1-f(1)&0&0&0&0\\
0&0&0&0&1-f(2)&f(2)&0&0\\
0&0&0&0&f(2)&1-f(2)&0&0\\
0&0&0&0&0&0&1-f(3)&f(3)\\
0&0&0&0&0&0&f(3)&1-f(3)\\

\end{matrix}
$$ 

Notese que entonces tenemos 2 Qbits y que la dimencion de nuestra matris es $2^{2+1}=2^{3}=8$, sumado a esto vemos  un patron muy claro en la matriz que se produce , con una *diagonal en bloques* que puede ser expandible a cualquier valor de $n$

Con esto dicho , podemos comenzar con la programacion : 

In [1]:
import pennylane as qml
import numpy as np    

Ahora requerimos ingresar un numero natural que sea nuestra cantidad de Qbits $|0\rangle$  : 

In [21]:
n = int(input("Por favor, ingresa un número natural: "))

In [20]:
#Crea el dispositivo cuantico
dev = qml.device('default.qubit', wires=n+1,shots=5)

#La numeracion de los wires comienza en 0 y llegara hasta el wire n, haciendo un total de n+1 wires

Definiremos ahora las funciones posibles que puede tomar $f(x)$  , que  seran analogas a las funciones *identidad*,*flip* y las funciones *constantes*; que trabajarian con el modulo 2 de el numero $x$ que se  ingrese en estas: 

In [4]:
#Funcion 1. Identidad 
 
def f1(x):
  #f:{0,1}-->{0,1}
  if (x%2)==0:
    return 0
  if (x%2)==1:
    return 1
  
#Funcion 2. Flip 
 
def f2(x):
  #f:{0,1}-->{0,1}
  if (x%2)==0:
    return 1
  if (x%2)==1:
    return 0
  
#Funcion 3. Constante 0 
 
def f3(x):
  #f:{0,1}-->{0,1}
  if (x%2)==0:
    return 0
  if (x%2)==1:
    return 0 
  
#Funcion 4. Constante 1 
 
def f4(x):
  #f:{0,1}-->{0,1}
  if (x%2)==0:
    return 1
  if (x%2)==1:
    return 1

Preparar el oráculo por el que pasarán nuestros qubits. Para esto formamos una matris respecto con las nesecidades del vector  original .

In [5]:

dim = 2**(n+1) 

print(f"Generamos la matriz Uf  de la dimencion requerida  {dim}x{dim}:") 

def U(g):
  u=np.zeros((dim, dim)) 
  # Generar la matriz de ceros de tamaño 2^(n+1) x 2^(n+1)

  for i in range(dim):
    if (i%2)==0:
      u[i,i]=1-g(i/2)
      u[i,i+1]=g(i/2)
      u[i+1,i]=g(i/2)
    else:
      u[i,i]=1-g((i-1)/2) 

  return u






Generamos la matriz Uf  de la dimencion requerida  32x32:


Generamos el circuito : 


In [65]:
@qml.qnode(dev)
def circuit(L): #Aqui L es un Operador Uf con algun f albitrairo 

#Como todos los atomos se incializan con estados |0> , a nuestro ultimo Qbit lo hacemos |1>
  qml.PauliX(wires=n)

# Aplicar Hadamard desde wire 0 hasta wire n
  for i in range(0,n):
      qml.Hadamard(wires=i)

# Aplicamos  Hadamard al ultimo qbit , que ahora es |1>, para que ahora sea |->
 # qml.Hadamard(wires=n)

#Oraculo
  qml.QubitUnitary(L,wires=list(range(n+1)))  
  #Queremos que todos los Qbits, que son n+1 , sean transformados por el operador que se hizo. 

#Aplicamos Hadamard  a los primeros n Qbits de nuevo 
  for i in range(n):
      qml.Hadamard(wires=i)

#Medida
  return qml.sample(wires=list(range(n)))

Ahora estamos listos para  ejecutar nuestros circuitos en funcion de las diferentes posibles funciones $f(x)$ que podemos tener.  

* **Circuito 1: Funcion *identidad***

In [69]:
circuit(U(f1))

array([[0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=int64)

* **Circuito 2: Funcion *flip***

In [70]:
circuit(U(f2))

array([[0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 1],
       [0, 0, 0, 0]], dtype=int64)

* **Circuito 3: Funcion *Constante 0***

In [71]:
circuit(U(f3))

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=int64)

* **Circuito 4: Funcion *Constante 1***

In [50]:
circuit(U(f3))

array([[0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 1]], dtype=int64)

Los resultados medidos tienen dos casos : 

* **Caso Constante** : El *array* regresado constara **unicamente** de ceros. 

* **Caso Balanceado** : El *array* regresado contendra **al menos**  un uno dentro de el.

**Pregunta** 

¿Por que es que varian cada una de las mediciones? ¿Es por el simulador cuantico? 

¿Como puedo manejar esos datos para que pueda decidir si efectivamente mi funcion es constante o es  Balanceada?